# CSC324 Lecture 6

**Disclaimer**:  Most of the notes were taken from [*Learn you a Haskell*](http://learnyouahaskell.com/a-fistful-of-monads) and [*Computerphile's Video*](https://www.youtube.com/watch?v=t1e8gqXLbsU&t=4s)

Last lecture, we talked about functors (`fmap`: mapping of the form `f a (a -> b) -> f b`).  
We also talked about applicative functors (`<*>`: mapping of the form `f a -> f (a -> b) -> f b`).  
Today, we will talk about `Monads` (it contains `>>=`: mapping the of the form `f a -> (a -> f b) f b`)

## Monads

### Motivation

Consider the following function below. 

In [1]:
import Data.Maybe as Maybe
-- monadicFunction :: a -> Maybe a
monadicFunction x = Just (x + 1)
:t monadicFunction

monadicFunction :: forall a. Num a => a -> Maybe a

Notice that unlike the functions we described in previous lectures, the monadicFunction takes a normal number and returns a data type (`Maybe` in the above example).

In [2]:
monadicFunction 3
b = monadicFunction 5
:t b

Just 4

b :: forall a. Num a => Maybe a

In [3]:
applyMaybe (Just 3) monadicFunction

<interactive>:1:1: error: Variable not in scope: applyMaybe :: Maybe Integer -> (Integer -> Maybe Integer) -> t

Now suppose I have a value, but it is wrapped in a `Just`. That is suppose, I have `Just 14` and I want to apply the monadicFunction to get `Just 15`. The issue is that I can't just pass in the `Just 14` as the function takes the number `14` instead of the entire `Maybe`, `Just 14`.

In [4]:
monadicFunction (Just 3) 

<interactive>:1:1: error:
    • Non type-variable argument in the constraint: Num (Maybe a)
      (Use FlexibleContexts to permit this)
    • When checking the inferred type
        it :: forall a. (Num a, Num (Maybe a)) => Maybe (Maybe a)

Let's define a function that can handle this 

In [5]:
applyMaybe :: Maybe a -> (a -> Maybe b) -> Maybe b   -- Notice the signature is exactly what we want
-- The second argument is of the type similar to the monadicFunction we described above
applyMaybe Nothing f = Nothing -- We don't do anything on Nothing
applyMaybe (Just x) f = f x -- We apply f on that value

In [6]:
applyMaybe (Just 3) monadicFunction

Just 4

Yaay that worked! We essentially uncovered what monads are. Similar to functors and applicative functors, a monad is a typeclass that has a function (**binder**) that takes in a *Monad* and feeds it to a function that takes in a value and returns a *Monad*  
In the above example, `applyMaybe` is considered to be the **binder** (`>>=` symbol) to the monad.


### The Monad Typeclass

Here is what a `Monad` typeclass looks like:

```
class Monad m where
    return :: a -> m a
    
    (>>=) :: m a -> (a -> m b) -> m b
    (>>) :: m a -> m b -> m b
    
    x >> y = x >>= \_ -> y 
    
    fail :: String -> m a
    fail msg = error msg
    

```

Notice that `return` is very similar to `pure` in Functors. As a matter of fact, they do the same thing. The functions `>>` and `fail` are implemented defaultly by Haskell and as long as you have `pure`, `return` is also implemented by default. Thus, you must have an instance of `Applicative Functor` before writing for `Monad` class. The main function that you have to implement is `>>=` aka the bind function. Here's an example to the `Maybe` class

In [7]:
data OurMaybe x = OurNothing | OurJust x

In [8]:
instance Functor OurMaybe where
    fmap f OurNothing = OurNothing
    fmap f (OurJust x) = OurJust (f x)

instance Applicative OurMaybe where
    pure = OurJust
    b <*> OurNothing = OurNothing
    (OurJust f) <*> (OurJust x) = OurJust (f x)

In [9]:
instance Monad OurMaybe where
    return = OurJust
    OurNothing >>= f = OurNothing
    (OurJust x) >>= f = f x

### Why Monads ?
Monads are very powerful. It may seem like applicative functors are stronger than monads, since applicative functors allow us to take a normal function and make it operate on values with contexts. We'll see that monads can do that as well because they're an upgrade of applicative functors, and that they can also do some cool stuff that applicative functors can't.

Consider the datatype `Frac` defined by the following:

In [11]:
data Frac x = Val x | Div (Frac x) (Frac x) 

It represents the fractions. Either just a number $x$ if $x \in \mathbb{Z}$ or a fraction $\frac{x}{y}$ where $x, y \in $ `Frac`

Consider the function `eval x` that takes in `Frac x` as an argument. It evaluates the representation to a number. It is easy to consider the case for `Val x` since we just return the number x

In [12]:
eval :: Frac x -> x
eval (Val x) = x

However, for the case of an actual fraction, we need to ensure that the denominator is not 0. Since it won't be possible to evaluate it then. 

```
eval (Div x y) = x `safeDiv` y -- We will define safeDive
```

In [13]:
safeDiv :: Int -> Int -> Maybe Int
safeDiv _ 0 = Nothing
safeDiv a b = Just (a `div` b)

Now we can evaluate `eval (Div x y)` right?? right ???  
Well, it looks like we're taking advantage of returning `Nothing` instead of raising an Exception for `safeDiv` so we decided to use `Maybe`. But do you see the issue when we just do `safeDiv x y` here?

How about we return `Maybe` for eval instead of the actual value? Sounds good!  
So can we just apply div Now ? Not really... We need to check if the second argument is `Nothing`

In [14]:
-- eval :: Frac -> Maybe
eval (Val x) = Just x
eval (Div x y) = case eval x of 
    Nothing -> Nothing
    Just n -> case eval y of 
        Nothing -> Nothing
        Just m -> safeDiv n m

In [15]:
a = Val 3
b = Val 4
c = Val 0
eval b
eval (Div b a) -- Evaluate 4 / 3
eval (Div b c) -- Evaluate 4 / 0

Just 4

Just 1

Nothing

Wow that was complicated. Now, if you show this to someone, he/she will have a hard time understand what this simple function does... 

This is where we can use a `Monad` for `Maybe`!  
Notice that `safeDiv a` has a signature that is similar to the function in the binder for a monad.  
We can exploit that by passing `eval x` (Which is `Maybe`) to safeDiv that has been evaluated partially.  
We do that using nested lambda expressions which should be familiar by now.

In [16]:
eval (Val x) = return x
eval (Div x y) = eval x >>= (\n ->
                 eval y >>= (\m ->
                 safeDiv n m))

Line 3: Avoid lambda
Found:
\ m -> safeDiv n m
Why not:
safeDiv n

#### The `do` notation

The above function can actually be simplified further using the `do` expression. But what is a `do` expression?  
The `do` notation is a special syntax for Monads for *gluing* monadic things together.  
In other words it helps to make nested monads look neat

If you have a nested expression of the form 
```
    -- There is a function that takes in 2 arguments values and returns a maybe
    function :: a -> b -> Maybe c
    Maybe a >>= (\arg1 -> -- Apply a Monad on a lambda that takes in one value (arg1).
                          -- Note that the function is of type (b -> Maybe c). So Monad works here
    Maybe b >>= (\arg2 -> -- This lambda applies Monad on a lambda that takes in one value (arg2)
                          -- And then we can evaluate function which returns Maybe
    function arg1 arg2))
```
This nested expression can keep on going for n arguments... You get the idea..  
This can be simplified to the following:  
```
do 
    arg1 <- Maybe a
    arg2 <- Maybe b
    function arg1 arg2
```
The above 2 are equivalent


Thus if you have an expression of the form

In [17]:
foo :: Maybe String  
foo = Just 3   >>= (\x -> 
      Just "!" >>= (\y -> 
      Just (show x ++ y)))  

It can be simplified to this

In [18]:
foo :: Maybe String  
foo = do  
    x <- Just 3  
    y <- Just "!"  
    Just (show x ++ y)  

Thus, our original function `eval (Div x y)` can be simplified further to just 3 simple understandable lines!

In [19]:
eval (Div x y) = do 
                n <- eval x -- Get Numerator
                m <- eval y -- Get Denominator
                safeDiv n m -- Evaluate division by safely dividing

Now even a baby can understand this! 